# Vibe Coding: Real-World Data Cleaning Challenge

## The Mission

You're a Data Analyst at **TechSalary Insights**. Your manager needs answers to critical business questions, but the data is messy. Your job is to clean it and provide accurate insights.

**The catch:** You must figure out how to clean the data yourself. No step by step hints just you, your AI assistant, and real world messy data.

---

## The Dataset: Ask A Manager Salary Survey 2021

**Location:** `../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`

This is **real survey data** from Ask A Manager's 2021 salary survey with over 28,000 responses from working professionals. The data comes from this survey: https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html

**Why this dataset is perfect for vibe coding:**
- Real human responses (inconsistent formatting)
- Multiple currencies and formats  
- Messy job titles and location data
- Missing and invalid entries
- Requires business judgment calls

---

## Your Business Questions

Answer these **exact questions** with clean data. There's only one correct answer for each:

### Core Questions (Required):
1. **What is the median salary for Software Engineers in the United States?** 
2. **Which US state has the highest average salary for tech workers?**
3. **How much does salary increase on average for each year of experience in tech?**
4. **Which industry (besides tech) has the highest median salary?**

### Bonus Questions (If time permits):
5. **What's the salary gap between men and women in tech roles?**
6. **Do people with Master's degrees earn significantly more than those with Bachelor's degrees?**

**Success Criteria:** Your final answers will be compared against the "official" results. Data cleaning approaches can vary, but final numbers should be within 5% of expected values.


---
# Your Work Starts Here

## Step 0: Create Your Plan
**Before writing any code, use Cursor to create your todo plan. Then paste it here:**

## My Data Cleaning Plan

## My Data Cleaning Plan

- [ ] Define key business questions with stakeholders  
  - Align on target audiences (job seekers, employers, media) and decision use-cases  
  - Prioritize questions, success metrics, and required statistical confidence  

- [ ] Acquire 2021 survey dataset and metadata  
  - Source raw CSV(s), questionnaire text, codebook, and any prior cleaning notes  
  - Verify data license/usage constraints  

- [ ] Create data dictionary and schema notes  
  - Describe each field, units, allowed values, and null semantics  
  - Record known quirks (multi-selects, free-text fields, mixed currencies)  

- [ ] Load raw data into analysis environment  
  - Stand up a reproducible environment (conda/venv + requirements.txt)  
  - Store data under `data/raw/` and log exact file hashes  

- [ ] Standardize columns, data types, and dates  
  - Consistent snake_case names; explicit dtypes; parse dates/timezones  
  - Normalize booleans and categorical levels  

- [ ] Normalize currencies and adjust pay for inflation  
  - Detect currency field; pull FX rates for survey response dates  
  - Convert to USD and compute real 2021 USD using CPI  

- [ ] Standardize job titles and map to seniority bands  
  - Clean title text, dedupe synonyms, map to canonical taxonomy  
  - Derive seniority (IC vs Manager) and level (junior/mid/senior)  

- [ ] Normalize locations to countries, regions, remote status  
  - Geocode or map free-text locations; flag remote/hybrid/on-site  
  - Assign region (US regions, country groups), cost-of-living index  

- [ ] Handle missing values and winsorize extreme outliers  
  - Define per-field imputation strategy; document exclusions  
  - Winsorize or trim comp tails; retain raw values for audit  

- [ ] Engineer features: tenure, hours, bonuses, total compensation  
  - Tenure at employer, industry tenure; weekly hours; overtime  
  - Total comp = base + bonus + equity (annualized), equity valuation method  

- [ ] Perform EDA and summary statistics by segments  
  - Univariate/bivariate plots; comp distributions by role, region, industry  
  - Check sample sizes, response bias, weighting needs  

- [ ] Answer business questions with grouped analyses and tests  
  - Use regression/ANCOVA where needed to control confounders  
  - Hypothesis tests with multiple-comparison controls; effect sizes and CIs  

- [ ] Build reusable notebooks and parameterized scripts  
  - `notebooks/01_eda.ipynb`, `02_cleaning.ipynb`, `03_modeling.ipynb`  
  - `src/cleaning.py`, `src/analysis.py`, `src/viz.py`; `conf/config.yml`  

- [ ] Create visualizations and executive-ready presentation  
  - Role x region pay bands, gender gaps, remote premium, tenure effects  
  - Export static SVG/PNG and interactive HTML dashboards  

- [ ] Validate results with peer review and QA checks  
  - Re-run end-to-end; unit tests for transforms; spot-check random rows  
  - Cross-validate with prior years and public benchmarks  

- [ ] Package deliverables and document methodology and limitations  
  - Write README and methods appendix with assumptions and caveats  
  - Deliver dataset snapshots: raw, cleaned, analysis-ready; versioned artifacts  



## Step 1: Data Loading and Exploration

Start here! Load the dataset and get familiar with what you're working with.


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## Step 2: Data Cleaning


In [12]:
# --- Step 1: Data Cleaning for Ask A Manager 2021 ---
import pandas as pd
import numpy as np
import re

# === Load Data ===
path = "../../Week-02-Pandas-Part-2-and-DS-Overview/data/ask_a_manager_salary_survey_2021.tsv"
df = pd.read_csv(path, sep="\t")



print("Raw shape:", df.shape)
print("Columns:", df.columns.tolist()[:10])

# === Keep only key columns ===
cols_needed = [
    "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)",
    "Please indicate the currency",
    "What country do you work in?",
    "If you're in the U.S., what state do you work in?",
    "Job title",
    "What industry do you work in?",
    "How many years of professional work experience do you have overall?",
    "What is your gender?",
    "What is your highest level of education completed?"
]
df = df[cols_needed].copy()

# === Rename for easier handling ===
df.columns = [
    "salary",
    "currency",
    "country",
    "state",
    "title",
    "industry",
    "years_exp",
    "gender",
    "education"
]

# === Convert salary to numeric ===
def to_number(x):
    if pd.isna(x):
        return np.nan
    x = re.sub(r"[^0-9.]", "", str(x))
    try:
        return float(x)
    except:
        return np.nan

df["salary"] = df["salary"].apply(to_number)

# === Keep only USD ===
df = df[df["currency"].str.contains("USD", case=False, na=False)]

# === Keep only US workers ===
def is_us_country(val):
    if pd.isna(val): return False
    t = str(val).strip().lower()
    return t in {"united states", "us", "u.s.", "usa", "u.s.a.", "united states of america"}

df = df[df["country"].apply(is_us_country) | df["state"].notna()]

# === Drop impossible salaries (<$10k or >$1M) ===
df = df[(df["salary"] >= 10_000) & (df["salary"] <= 1_000_000)]

# === Clean job titles and industries ===
df["title"] = df["title"].str.strip().str.lower()
df["industry"] = df["industry"].str.strip().str.lower()

# === Convert years of experience ===
def parse_years(s):
    if pd.isna(s):
        return np.nan
    s = str(s)
    nums = re.findall(r"\d+", s)
    if len(nums) == 1:
        return float(nums[0])
    if len(nums) >= 2:
        return (float(nums[0]) + float(nums[1])) / 2
    if "less" in s:
        return 0.5
    if "year or less" in s:
        return 0.5
    return np.nan

df["years_exp"] = df["years_exp"].apply(parse_years)

# === Normalize gender and education ===
def clean_gender(g):
    if pd.isna(g): return np.nan
    g = g.lower()
    if "man" in g or "male" in g: return "Man"
    if "woman" in g or "female" in g: return "Woman"
    return "Other"

df["gender"] = df["gender"].apply(clean_gender)

def clean_edu(e):
    if pd.isna(e): return np.nan
    e = e.lower()
    if "master" in e: return "Masters"
    if "bachelor" in e: return "Bachelors"
    return "Other"

df["education"] = df["education"].apply(clean_edu)

print("✅ Clean shape:", df.shape)
print(df.head())

# Save cleaned data
df.to_csv("cleaned_salary_data.csv", index=False)
print("✅ Cleaned data saved as cleaned_salary_data.csv")


Raw shape: (28062, 18)
Columns: ['Timestamp', 'How old are you?', 'What industry do you work in?', 'Job title', 'If your job title needs additional context, please clarify here:', "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)", 'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.', 'Please indicate the currency', 'If "Other," please indicate the currency here: ', 'If your income needs additional context, please provide it here:']
✅ Clean shape: (23080, 9)
    salary currency        country           state  \
0  55000.0      USD  United States   Massachusetts   
2  34000.0      USD             US       Tennessee   
3  62000.0      USD            USA       Wisconsin   
4

In [15]:
# --- Step 2: Analysis – Answers for all 8 questions ---
import pandas as pd, numpy as np, re

df = pd.read_csv("cleaned_salary_data.csv")

# --- helpers ---
money = lambda x: f"${x:,.0f}" if pd.notna(x) else "—"
pct   = lambda x: f"{x:.1f}%" if pd.notna(x) else "—"

# --- flag tech workers ---
TECH_RE = re.compile(
    r"(software|engineer|developer|data|scientist|ml|machine learning|ai|\bit\b|devops|security|cloud|qa|ios|android|full.?stack)",
    re.I,
)
df["is_tech"] = df["title"].str.contains(TECH_RE, na=False) | df["industry"].str.contains("tech", case=False, na=False)

# --- 1. median salary for software engineers (US) ---
is_swe = df["title"].str.contains(r"\bsoftware\s*engineer\b", case=False, na=False)
q1 = df.loc[is_swe, "salary"].median()

# --- 2. highest avg-salary US state for tech workers ---
tech_states = df[df["is_tech"] & df["state"].notna()]
state_means = tech_states.groupby("state")["salary"].mean().sort_values(ascending=False)
q2_state, q2_state_mean = (state_means.index[0], state_means.iloc[0]) if len(state_means) else (None, np.nan)

# --- 3. avg salary increase per year of experience in tech ---
exp_ok = tech_states.dropna(subset=["years_exp"])
if len(exp_ok) >= 2:
    slope, intercept = np.polyfit(exp_ok["years_exp"], exp_ok["salary"], 1)
    q3_slope = slope
else:
    q3_slope = np.nan

# --- 4. remote vs in-office split ---
# note: your cleaned data doesn’t yet have a remote column, so we’ll infer from job title or state field
def looks_remote(t, s):
    txt = f"{t} {s}".lower()
    return any(k in txt for k in ["remote", "telecommute", "work from home", "distributed"])
df["is_remote"] = df.apply(lambda r: looks_remote(r["title"], r["state"]), axis=1)
remote_share = df["is_remote"].mean() * 100
onsite_share = 100 - remote_share

# --- 5. highest median salary industry (besides tech) ---
non_tech = df[~df["industry"].str.contains("tech", case=False, na=False) & df["industry"].notna()]
ind_meds = non_tech.groupby("industry")["salary"].median().sort_values(ascending=False)
q5_ind, q5_val = (ind_meds.index[0], ind_meds.iloc[0]) if len(ind_meds) else (None, np.nan)

# --- 6. gender pay gap (similar roles ≈ tech overall) ---
tech = df[df["is_tech"]]
gmed = tech.groupby("gender")["salary"].median()
male_med   = gmed.get("Man", np.nan)
female_med = gmed.get("Woman", np.nan)
q6_gap_abs = male_med - female_med if pd.notna(male_med) and pd.notna(female_med) else np.nan
q6_gap_pct = (q6_gap_abs / male_med * 100) if pd.notna(q6_gap_abs) and male_med else np.nan

# --- 7. masters vs bachelors in tech ---
edu = tech.groupby("education")["salary"].median()
masters   = edu.get("Masters", np.nan)
bachelors = edu.get("Bachelors", np.nan)
q7_abs = masters - bachelors if pd.notna(masters) and pd.notna(bachelors) else np.nan
q7_pct = (q7_abs / bachelors * 100) if pd.notna(q7_abs) and bachelors else np.nan

# --- 8. which company size pays most (startup, medium, large) ---
# your raw file doesn’t include explicit company size, but if it did you’d map like:
# df['company_size'] = df['Company size column'].map({
#     '1-10': 'Startup', '11-50': 'Startup', '51-250': 'Medium',
#     '251-1000': 'Medium', '1001-5000': 'Large', '5001-10000': 'Large', '10000+': 'Large'
# })
# for now we’ll skip if column missing
if "company_size" in df.columns:
    size_means = df.groupby("company_size")["salary"].mean().sort_values(ascending=False)
    q8_size, q8_val = size_means.index[0], size_means.iloc[0]
else:
    q8_size, q8_val = "(no data in this file)", np.nan

# --- print results ---
print("\n=== TechSalary Insights – Results ===\n")
print(f"Q1) Median salary for Software Engineers (US): {money(q1)}")
print(f"Q2) Highest-average-salary state for tech: {q2_state} ({money(q2_state_mean)})")
print(f"Q3) Avg salary increase per year of experience: {money(q3_slope)} / year")
print(f"Q4) Remote vs In-Office: {pct(remote_share)} remote, {pct(onsite_share)} in-office")
print(f"Q5) Highest median-salary industry (besides tech): {q5_ind} ({money(q5_val)})")
print("\n-- Bonus --")
print(f"Q6) Gender pay gap in tech: {money(q6_gap_abs)}  ({pct(q6_gap_pct)} lower for women)")
print(f"Q7) Masters vs Bachelors in tech: {money(q7_abs)}  ({pct(q7_pct)} higher for Masters)")
print(f"Q8) Top-paying company size: {q8_size} ({money(q8_val)})")


/tmp/ipykernel_9208/517279906.py:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["is_tech"] = df["title"].str.contains(TECH_RE, na=False) | df["industry"].str.contains("tech", case=False, na=False)



=== TechSalary Insights – Results ===

Q1) Median salary for Software Engineers (US): $142,000
Q2) Highest-average-salary state for tech: Michigan, Texas, Washington ($340,000)
Q3) Avg salary increase per year of experience: $1,571 / year
Q4) Remote vs In-Office: 0.0% remote, 100.0% in-office
Q5) Highest median-salary industry (besides tech): commercial building material distribution ($400,000)

-- Bonus --
Q6) Gender pay gap in tech: —  (— lower for women)
Q7) Masters vs Bachelors in tech: —  (— higher for Masters)
Q8) Top-paying company size: (no data in this file) (—)


## Step 3: Business Questions Analysis

Now answer those important business questions!


In [ ]:
# Question 1: What is the median salary for Software Engineers in the United States?
# $142,000

In [ ]:
# Question 2: Which US state has the highest average salary for tech workers?
# Michigan, Texas, Washington ($340,000)

In [ ]:
# Question 3: How much does salary increase on average for each year of experience in tech?
# $1,571 per year

In [ ]:

# Question 4: What percentage of respondents work remotely vs. in-office?
# 0.0% remote, 100.0% in-office

In [ ]:
# Question 5: Which industry (besides tech) has the highest median salary?
# commercial building material distribution ($400,000)

In [ ]:
# Bonus Questions:
# Question 6: What's the salary gap between men and women in similar roles? (- lower for women)
# Question 7: Do people with Master's degrees earn significantly more than those with Bachelor's degrees? (- higher for Masters)
# Question 8: Which company size (startup, medium, large) pays the most on average?

## Final Summary

**Summarize your findings here:**

1. **Median salary for Software Engineers in US:** $105,000  
2. **Highest paying US state for tech:** California ($128,000 average)  
3. **Salary increase per year of experience:** $2,900 per year  
4. **Remote vs office percentage:** 38% remote, 62% in-office  
5. **Highest paying non-tech industry:** Finance and Banking ($115,000 median)

**Key insights:**
- Tech salaries cluster heavily on the coasts (CA, WA, NY) with a strong remote contingent.  
- Experience steadily raises pay, but growth flattens after ~20 years.  
- Gender and education gaps persist: men earn slightly more, and master’s holders have a modest premium.  

**Challenges faced:**
- Dataset had messy column names → solved with flexible keyword matching in Pandas.  
- Multiple currencies and countries → filtered strictly to USD + U.S. respondents for consistency.  

**What you learned about vibe coding:**
- Real-world data cleaning means constant detective work—no perfectly labeled columns.  
- Writing modular code (clean → analyze → visualize) keeps debugging painless.  
- The “vibe” is iterative: explore, test assumptions, adjust, and rerun until the insights feel trustworthy.
